<a href="https://colab.research.google.com/github/luismiguelmartinluengo/Test_in_Colab/blob/main/Introduction_PySpark_SalesStore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports, variables y creación de la sesión

In [47]:
import pyspark
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, FloatType
from pyspark.sql.window import Window
import os
import math

In [ ]:
ruta_fichero = 'drive/MyDrive/Colab Notebooks/Ficheros de prueba/Superstore.csv'

In [ ]:
ss = SparkSession.builder.appName("MiAppSpark").getOrCreate()

El data frame tiene 9994 filas
+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|  Customer Name|  Segment|      Country|           City|     State|Postal Code|Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|     1|CA-2016-152156| 11/8/2016|11/11/2016|  Second Class|   CG-12520|    Claire Gute| Consumer|United States|      Henderson|  Kentucky|      42420| South|FUR-BO-10001798|      Furnit

## Carga de datos y adaptación del esquema

In [11]:
df = ss.read.csv(ruta_fichero, header=True, inferSchema=True)
print('El data frame tiene {} filas'.format(df.count()))
df.show(5)

El data frame tiene 9994 filas
+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|  Customer Name|  Segment|      Country|           City|     State|Postal Code|Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|     1|CA-2016-152156| 11/8/2016|11/11/2016|  Second Class|   CG-12520|    Claire Gute| Consumer|United States|      Henderson|  Kentucky|      42420| South|FUR-BO-10001798|      Furnit

In [31]:
df = df.withColumns({"Quantity":col("Quantity").cast(IntegerType()),
                     "Sales":col("Sales").cast(FloatType())})
df.printSchema()

root
 |-- Row ID: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Postal Code: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: float (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Discount: string (nullable = true)
 |-- Profit: double (nullable = true)



## Análisis

**Categoría con más unidades vendidas**

In [14]:
ventas_por_categoria = df.groupBy("Category").agg(sum("Quantity").alias("Total Quantity"))
categoria_mas_ventas = ventas_por_categoria.orderBy(col("Total Quantity").desc()).first()
print(f"'{categoria_mas_ventas['Category']}' es la categoría con más ventas, con un total de {categoria_mas_ventas['Total Quantity']} unidades vendidas")

'Office Supplies' es la categoría con más ventas, con un total de 36739 unidades vendidas


**Categoría con mayores ingresos**

In [24]:
ingresos_por_categoria = df.groupBy("Category").agg(sum("Sales").alias("Total Sales"))
categoria_mayor_ingresos = ingresos_por_categoria.orderBy(col("Total Sales").desc()).limit(1)
categoria_mayor_ingresos = categoria_mayor_ingresos.withColumn("Total Sales", round(categoria_mayor_ingresos["Total Sales"],2)).first()
print(f"'{categoria_mayor_ingresos['Category']}' es la categoría con más ingresos, con un total de {categoria_mayor_ingresos['Total Sales']} (moneda) ingresados")

'Technology' es la categoría con más ingresos, con un total de 835900.07 (moneda) ingresados


**Categoría con mayor beneficio**

In [25]:
# prompt: Mismo cálculo que la celda anterior pero para calcular los beneficios basados en la columna profit

beneficios_por_categoria = df.groupBy("Category").agg(sum("Profit").alias("Total Profit"))
categoria_mayor_beneficios = beneficios_por_categoria.orderBy(col("Total Profit").desc()).limit(1)
categoria_mayor_beneficios = categoria_mayor_beneficios.withColumn("Total Profit", round(categoria_mayor_beneficios["Total Profit"],2)).first()
print(f"'{categoria_mayor_beneficios['Category']}' es la categoría con más beneficios, con un total de {categoria_mayor_beneficios['Total Profit']} (moneda) de beneficio")


'Technology' es la categoría con más beneficios, con un total de 145388.3 (moneda) de beneficio


**Top 10 clientes con mayores ventas**

In [43]:
ventas_x_cliente = df.groupBy("Customer Name").agg(sum("Sales").alias("Total Sales"))\
                           .orderBy('Total Sales', ascending = False)\
                           .limit(10)\
                           .withColumn("Total Sales", round(col("Total Sales"),2))
ventas_x_cliente.show()


+------------------+-----------+
|     Customer Name|Total Sales|
+------------------+-----------+
|       Sean Miller|   25043.05|
|      Tamara Chand|   19017.85|
|      Raymond Buch|   15117.34|
|      Tom Ashbrook|   14595.62|
|     Adrian Barton|   14355.61|
|      Sanjit Chand|   14142.33|
|      Ken Lonsdale|   14071.92|
|      Hunter Lopez|    12873.3|
|      Sanjit Engle|   12209.44|
|Christopher Conant|   12129.07|
+------------------+-----------+



**Categoría con más unidades vendidas para cada tipo de cliente**

In [48]:
cat_seg_quantity = df.groupBy("Segment", "Category").agg(sum("Quantity").alias("Total Quantity"))
particiones_segmento = Window.partitionBy("Segment").orderBy(col("Total Quantity").desc())
cat_seg_quantity_max = cat_seg_quantity.withColumn("rank", rank().over(particiones_segmento)) \
                                        .filter(col("rank") == 1) \
                                        .drop("rank")
cat_seg_quantity_max.show()

+-----------+---------------+--------------+
|    Segment|       Category|Total Quantity|
+-----------+---------------+--------------+
|   Consumer|Office Supplies|         17122|
|  Corporate|Office Supplies|         13110|
|Home Office|Office Supplies|          6507|
+-----------+---------------+--------------+

